In [85]:
#библиотеки

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as ps
import plotly.graph_objects as go
import scipy
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

import statsmodels.api as sm


df = pd.read_csv("test.csv", sep=",")
df = df.drop(columns=['pc'], axis=1)
df.head()

,id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,1,1043,1,1.8,1,14,0,5,0.1,193,3,226,1412,3476,12,7,2,0,1,0
1,2,841,1,0.5,1,4,1,61,0.8,191,5,746,857,3895,6,0,7,1,0,0
2,3,1807,1,2.8,0,1,0,27,0.9,186,3,1270,1366,2396,17,10,10,0,1,1
3,4,1546,0,0.5,1,18,1,25,0.5,96,8,295,1752,3893,10,0,7,1,1,0
4,5,1434,0,1.4,0,11,1,49,0.5,108,6,749,810,1773,15,8,7,1,0,1


In [86]:
#деление на обучающую и тестовую выборки
#df = df.groupby('int_memory')['battery_power'].mean().reset_index()
x = df.battery_power[100:220].to_numpy()
y = df.int_memory[100:220].to_numpy()

#список индексов, отсортированных по возрастанию x (battery_power)
sorted_indices = np.argsort(x)
x = x[sorted_indices]
y = y[sorted_indices]

# % на выборках
oby4 = round(0.8*(220-100))

x_train = x[:oby4]
y_train = y[:oby4]
x_test = x[oby4:]
y_test = y[oby4:]


print('Обучающая выборка:')
print(x_train)
print(y_train)
print('\nТестовая выборка:')
print(x_test)
print(y_test)

Обучающая выборка:
[ 500  504  517  541  556  557  559  560  562  564  574  586  591  597
  607  624  639  651  683  697  723  725  758  762  769  796  800  852
  859  861  885  890  895  895  915  923  945  948  951  955  976  987
  988 1001 1004 1008 1012 1038 1039 1045 1051 1072 1081 1094 1140 1153
 1180 1186 1202 1204 1249 1250 1251 1278 1300 1307 1319 1348 1363 1376
 1378 1384 1397 1429 1455 1466 1483 1483 1496 1526 1532 1535 1547 1571
 1573 1577 1582 1599 1605 1611 1629 1664 1698 1715 1735 1739]
[64 63 38 28 10 23 24 12 43 33 60 33 21 16 18 30 28 35  9 34 63 26 48 39
 30 36 41  3 58 40  3  9  9 23 10 10 47 60 24 62  5 20 12 48  8 15 32 48
 15 58 16  4 17 34 56 14 43 25 49 19 38 63 54 56 43  4  5 50  2 38 52 62
 19 33 16 57 38 61 42 39  7 37 50 44 24 55 33 31 51 25  2 25 21 11 21 51]

Тестовая выборка:
[1762 1762 1765 1770 1771 1773 1783 1795 1806 1807 1811 1832 1840 1857
 1859 1863 1901 1930 1933 1933 1939 1963 1981 1986]
[50 11 24 26 39 61 23 52 64  8 25 44 46 14 42 64 62 64 55 

In [87]:
#MHK

def func(x, c0, c1, c2): #функция для аппроксимации
    return c0 + c1 * x + c2 * x**2

popt, _ = curve_fit(func, x_train, y_train)
c0, c1, c2 = popt

# Построение аппроксимации и графика данных
y_education = func(x_train, c0, c1, c2)
fig = go.Figure()

# обучающие данные
fig.add_trace(go.Scatter(
    x = x_train,
    y = y_train,
    mode = 'lines',
    name = 'Обучающие данные',
    line=dict(color = '#00BFFF', width = 2))
)

# МНК
fig.add_trace(go.Scatter(
    x = x_train,
    y = y_education,
    mode = 'lines',
    name = 'МНК',
    line = dict(color = '#FF8C00', shape = 'spline', width = 2))
)

fig.update_xaxes(title_text="Ёмкость батареи (mAh)")
fig.update_yaxes(title_text="Оперативная память RAM (Mhz)")
fig.update_layout(title_text='График "Зависимость ёмкости батареи от тактовой частоты"', title_x =0.5)
fig.show()

In [88]:
#MHK

def func(x, c0, c1, c2): #функция для аппроксимации
    return c0 + c1 * x + c2 * x**2

popt, _ = curve_fit(func, x_test, y_test)
c0, c1, c2 = popt

# Построение аппроксимации и графика данных
y_education = func(x_test, c0, c1, c2)
fig = go.Figure()

# тестовые данные
fig.add_trace(go.Scatter(
    x = x_test,
    y = y_test,
    mode = 'lines',
    name = 'Тестовые данные',
    line=dict(color = '#00BFFF', width = 2))
)

# МНК
fig.add_trace(go.Scatter(
    x = x_test,
    y = y_education,
    mode = 'lines',
    name = 'МНК',
    line = dict(color = '#FF8C00', shape = 'spline', width = 2))
)

fig.update_xaxes(title_text="Ёмкость батареи (mAh)")
fig.update_yaxes(title_text="Оперативная память RAM (Mhz)")
fig.update_layout(title_text='График "Зависимость ёмкости батареи от тактовой частоты"', title_x =0.5)
fig.show()

In [89]:
#KNn

k = 2  # соседи
model = KNeighborsClassifier(n_neighbors=k)
model.fit(x_train.reshape(-1,1), y_train)
y_pred = model.predict(x_train.reshape(-1,1))
# обучающие данные
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = x_train,
    y = y_train,
    mode = 'lines',
    name = 'Обучающие данные',
    line=dict(color = '#4682B4', width = 2))
)

# МНК
fig.add_trace(go.Scatter(
    x = x_train,
    y = y_pred,
    mode = 'lines',
    name = 'KNn',
    line = dict(color = 'red', width = 2))
)
fig.update_xaxes(title_text="Ёмкость батареи (mAh)")
fig.update_yaxes(title_text="Оперативная память RAM (Mhz)")
fig.update_layout(title_text='График "Зависимость ёмкости батареи от тактовой частоты"', title_x =0.5)
fig.show()

In [91]:
#KNn

k = 2  # соседи
model = KNeighborsClassifier(n_neighbors=k)
model.fit(x_test.reshape(-1,1), y_test)
y_pred = model.predict(x_test.reshape(-1,1))
# тестовые данные
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = x_test,
    y = y_test,
    mode = 'lines',
    name = 'Тестовые данные',
    line=dict(color = '#4682B4', width = 2))
)

# МНК
fig.add_trace(go.Scatter(
    x = x_test,
    y = y_pred,
    mode = 'lines',
    name = 'KNn',
    line = dict(color = 'red', width = 2))
)
fig.update_xaxes(title_text="Ёмкость батареи (mAh)")
fig.update_yaxes(title_text="Оперативная память RAM (Mhz)")
fig.update_layout(title_text='График "Зависимость ёмкости батареи от тактовой частоты"', title_x =0.5)
fig.show()

In [107]:
#ARIMA

y_pred_arima=[]
y_train_save = []
for i in y_train:
    y_train_save.append(y_train[i]) #заполняем уже имеющимися

p = 0  # порядок авторегрессии
d = 1  # порядок интеграции
q = 0 # порядок скользящего среднего
for i in range(len(x_test)):
    model = sm.tsa.arima.ARIMA(y_train_save, order=(p, d, q)).fit()
    y_pred_arima.append(model.forecast()[0])
    y_train_save.append(y_train[i])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x = x,
    y = y,
    mode = 'lines',
    name = 'Значения',
    line=dict(color = '#4682B4', width = 2))
)

# МНК
fig.add_trace(go.Scatter(
    x = x_test,
    y = y_pred_arima,
    mode = 'lines',
    name = 'ARIMA',
    line = dict(color = 'red', width = 2))
)
fig.update_xaxes(title_text="Ёмкость батареи (mAh)")
fig.update_yaxes(title_text="Оперативная память RAM (Mhz)")
fig.update_layout(title_text='График "Зависимость ёмкости батареи от тактовой частоты"', title_x =0.5)
fig.show()